In [1]:
import pandas as pd

In [66]:
def asint(x):
    try:
        return int(x)
    except:
        return 0

gt_file = '../data/out/SemD/13428_2012_278_MOESM1_ESM.csv'
gt = pd.read_csv(gt_file, keep_default_na=False).sort_values('term')
gt = gt.set_index('term')
gt.BNC_contexts = gt.BNC_contexts.map(asint).astype(int)
gt

,mean_cos,SemD,BNC_wordcount,BNC_contexts,BNC_freq,lg_BNC_freq
term,,,,,,
aa,0.020000,1.69,577,314,6.6,0.88
aah,0.085245,1.07,92,58,1.1,0.31
aback,0.025864,1.59,294,293,3.4,0.64
abacus,0.022685,1.64,51,40,0.6,0.20
abandon,0.008199,2.09,1257,1193,14.4,1.19
...,...,...,...,...,...,...
zoom,0.051591,1.29,241,161,2.8,0.58
zoomed,0.029300,1.53,64,59,0.7,0.24
zooming,0.023874,1.62,61,54,0.7,0.23


In [53]:
[c for c in gt.columns if c in stats.columns]

['BNC_wordcount', 'BNC_contexts', 'BNC_freq', 'lg_BNC_freq']

In [54]:
stats_file = '../data/out/cache/SemD/entropy_transform.csv'
stats = pd.read_csv(stats_file, sep='\t', keep_default_na=False).sort_values('token')
stats = stats.rename(columns={
    'token': 'term', 
    'context_freq': 'BNC_contexts',
    'corpus_freq': 'BNC_wordcount',
    'freq': 'BNC_freq',
    'log_freq': 'lg_BNC_freq',
})
stats = stats[stats.term.isin(gt.index)]
stats = stats.set_index('term')
stats = stats[[c for c in gt.columns if c in stats.columns]]
stats

,BNC_wordcount,BNC_contexts,BNC_freq,lg_BNC_freq
term,,,,
aa,481,259,5.005948,0.778582
aah,219,124,2.279216,0.515770
aback,303,300,3.153435,0.618407
abacus,53,42,0.551591,0.190777
abandon,1294,1224,13.467146,1.160383
...,...,...,...,...
zoom,259,174,2.695511,0.567674
zoomed,66,61,0.686887,0.227086
zooming,69,62,0.718109,0.235051


In [78]:
semd_file = '../data/out/SemD/default/bnc_default.semd'
semd = pd.read_csv(semd_file, sep=',', keep_default_na=False, index_col=0)
semd.index.name = 'term'
semd = semd.rename(columns={'semd': 'SemD'})
semd = semd[semd.index.isin(gt.index)]
semd = semd.sort_index()
semd

,mean_cos,SemD
term,,
aa,0.594473,0.225868
aah,0.725614,0.139295
aback,0.722525,0.141147
abacus,0.654359,0.184184
abandon,0.609364,0.215123
...,...,...
zoom,0.605017,0.218232
zoomed,0.632578,0.198886
zooming,0.650754,0.186583


In [81]:
semd = semd.join(stats)

In [67]:
gt = gt[gt.index.isin(stats.index)]
gt

,mean_cos,SemD,BNC_wordcount,BNC_contexts,BNC_freq,lg_BNC_freq
term,,,,,,
aa,0.020000,1.69,577,314,6.6,0.88
aah,0.085245,1.07,92,58,1.1,0.31
aback,0.025864,1.59,294,293,3.4,0.64
abacus,0.022685,1.64,51,40,0.6,0.20
abandon,0.008199,2.09,1257,1193,14.4,1.19
...,...,...,...,...,...,...
zoom,0.051591,1.29,241,161,2.8,0.58
zoomed,0.029300,1.53,64,59,0.7,0.24
zooming,0.023874,1.62,61,54,0.7,0.23


In [89]:
x = pd.concat([gt.mean_cos, semd.mean_cos], axis=1)
x.columns = ['gt', 'new']
x

,gt,new
term,,
aa,0.020000,0.594473
aah,0.085245,0.725614
aback,0.025864,0.722525
abacus,0.022685,0.654359
abandon,0.008199,0.609364
...,...,...
zoom,0.051591,0.605017
zoomed,0.029300,0.632578
zooming,0.023874,0.650754


In [95]:
import numpy as np
(-np.log10(semd.mean_cos) - semd.SemD)

term
aa        -6.251570e-09
aah       -1.816196e-09
aback     -3.320372e-09
abacus    -3.399790e-09
abandon   -1.710589e-08
               ...     
zoom      -1.268550e-08
zoomed    -4.659572e-09
zooming   -6.111456e-10
zoos      -1.030749e-08
zulu      -6.411364e-09
Length: 31335, dtype: float64

In [83]:
gt.dtypes, semd.dtypes

(mean_cos         float64
 SemD             float64
 BNC_wordcount      int64
 BNC_contexts       int64
 BNC_freq         float64
 lg_BNC_freq      float64
 dtype: object,
 mean_cos         float64
 SemD             float64
 BNC_wordcount      int64
 BNC_contexts       int64
 BNC_freq         float64
 lg_BNC_freq      float64
 dtype: object)

In [84]:
semd.corrwith(gt, axis=0)

mean_cos         0.436173
SemD             0.460770
BNC_wordcount    0.997916
BNC_contexts     0.996900
BNC_freq         0.997916
lg_BNC_freq      0.994125
dtype: float64

In [33]:
a = dict(x=1, y=2, z=3)
b = dict(x=0.1, y=0.2, z=0.3)
pd.DataFrame([a, b], index=['mean_cos', 'semd']).T

,mean_cos,semd
x,1.0,0.1
y,2.0,0.2
z,3.0,0.3


In [19]:
gt.term.str.match(re_abc).sum()

31739